In [93]:
!pip install -q -U google-generativeai python-dotenv pandas Jinja2

I0000 00:00:1722553394.568100 6997231 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork


In [154]:
import google.generativeai as genai
from IPython.display import Markdown, Image
import os, json
from dotenv import load_dotenv
import pandas as pd

load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [155]:
actual_data = {
    "images/1.png": {
        "calories": 357.81,
        "protein": 20.00,
        "carb": 20.96,
        "fat": 21.69
    },
    "images/2.png": {
        "calories": 655.99,
        "protein": 93.24,
        "carb": 23.34,
        "fat": 20.22
    },
    "images/3.png": {
        "calories": 351.89,
        "protein": 12.10,
        "carb": 57.50,
        "fat": 9.77
    },
    "images/4.png": {
        "calories": 184.99,
        "protein": 24.54,
        "carb": 8.90,
        "fat": 6.14
    },
    "images/5.png": {
        "calories": 139.56,
        "protein": 5.40,
        "carb": 14.66,
        "fat": 6.64
    },
    "images/6.png": {
        "calories": 950.40,
        "protein": 74.77,
        "carb": 54.13,
        "fat": 45.30
    },
    "images/7.png": {
        "calories": 284.69,
        "protein": 13.84,
        "carb": 14.42,
        "fat": 18.80
    },
    "images/8.png": {
        "calories": 557.73,
        "protein": 25.77,
        "carb": 23.81,
        "fat": 42.54
    },
    "images/9.png": {
        "calories": 583.38,
        "protein": 42.29,
        "carb": 52.42,
        "fat": 23.33
    },
    "images/10.png": {
        "calories": 163.86,
        "protein": 2.51,
        "carb": 28.49,
        "fat": 5.30
    },
    "images/11.png": {
        "calories": 428.67,
        "protein": 24.47,
        "carb": 4.90,
        "fat": 35.14
    },
    "images/12.png": {
        "calories": 506.36,
        "protein": 77.39,
        "carb": 16.41,
        "fat": 13.48
    }
}

In [156]:
def get_predicted_data(image_path):
    # Upload the file
    sample_file = genai.upload_file(path=image_path)

    # Run Gemini model
    model = genai.GenerativeModel(model_name="gemini-1.5-pro")
    prompt = """
    You are an expert nutritionist with extensive experience in visual food analysis and portion estimation. Your task is to provide the most accurate nutrition information possible for the given food image.

    Instructions:
    1. Carefully analyze the image, considering that it was taken 50 cm above the food item.
    2. Identify all visible food items and ingredients in the image.
    3. For each identified food item and ingredient, get the per gram calories in kilocalories and macronutrients (protein, carbohydrates, fat) based on data from the USDA Food and Nutrient Database. 
    4. Estimate the weight of each food item and ingredient in grams, taking into account the known distance from the camera and visual cues.
    5. Calculate the nutritional content for each item by multiplying the per gram values from step 3 with the estimated weights from step 4.
    6. Consider the preparation method (e.g., fried, grilled, raw) as it affects nutritional content.
    7. Factor in any visible condiments, sauces, or dressings.
    8. Use your expertise to estimate the most likely recipe or preparation method if not immediately apparent.
    9. Round your estimates to the nearest whole number for simplicity.
    10. If you're uncertain about any aspect, provide your best estimate and indicate the level of confidence.

    Output your nutritional analysis in the following JSON format, ensuring all values are integers:
    {
        "items": [
            {
                "item": "<food_item>",
                "estimated_weight_g": <weight>,
                "per_100g": {
                    "calories": <calories>,
                    "protein": <protein>,
                    "carb": <carb>,
                    "fat": <fat>
                },
                "total": {
                    "calories": <calories>,
                    "protein": <protein>,
                    "carb": <carb>,
                    "fat": <fat>
                }
            },
            // Repeat for each food item
        ]
    }

    Remember, accuracy is crucial. Use all available visual cues and your nutritional expertise to provide the most precise estimates possible. Include the calculation steps for each ingredient for transparency and verification.
    """
    response = model.generate_content([sample_file, prompt])
    response_json = json.loads(response.text)

    return response_json

In [142]:
def format_predicted_data(predicted_data, image_path):
    # Create items DataFrame
    items_df = pd.DataFrame([
        {
            "Item": item["item"],
            "Estimated Weight (g)": item["estimated_weight_g"],
            "Calories": f"{item['total']['calories']} ({item['per_100g']['calories']}/100g)",
            "Protein (g)": f"{item['total']['protein']} ({item['per_100g']['protein']}/100g)",
            "Carbs (g)": f"{item['total']['carb']} ({item['per_100g']['carb']}/100g)",
            "Fat (g)": f"{item['total']['fat']} ({item['per_100g']['fat']}/100g)"
        }
        for item in predicted_data["items"]
    ])

    # Calculate totals
    totals = items_df[["Calories", "Protein (g)", "Carbs (g)", "Fat (g)"]].apply(lambda x: x.str.split().str[0].astype(float)).sum()

    # Create summary DataFrame
    summary_data = []
    for property, actual in actual_data[image_path].items():
        if property.lower() == "protein":
            predicted = totals["Protein (g)"]
        elif property.lower() == "carb":
            predicted = totals["Carbs (g)"]
        elif property.lower() == "fat":
            predicted = totals["Fat (g)"]
        else:
            predicted = totals[property.capitalize()]
        
        percent_error = abs((predicted - actual) / actual) * 100
        summary_data.append({
            "Nutrient": property.capitalize(),
            "Actual": round(actual, 2),
            "Predicted": round(predicted, 2),
            "Percent Error": round(percent_error, 2)
        })

    summary_df = pd.DataFrame(summary_data)

    # Style the DataFrames
    items_styled = items_df.style.set_caption("Food Items Breakdown").format({
        "Estimated Weight (g)": "{:.0f}",
        "Calories": "{}",
        "Protein (g)": "{}",
        "Carbs (g)": "{}",
        "Fat (g)": "{}"
    })

    summary_styled = summary_df.style.set_caption("Cumulative Nutritional Summary").format({
        "Actual": "{:.2f}",
        "Predicted": "{:.2f}",
        "Percent Error": "{:.2f}%"
    }).bar(subset=["Percent Error"], color="#FFA07A", vmin=0, vmax=100)

    return {
        "items": items_styled,
        "summary": summary_styled
    }

In [125]:
predicted_data_1 = get_predicted_data("images/1.png")
formatted_predicted_data_1 = format_predicted_data(predicted_data_1, "images/1.png")
display(formatted_predicted_data_1["items"])
display(formatted_predicted_data_1["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,Scrambled eggs,100,143 (143/100g),12 (12/100g),2 (2/100g),10 (10/100g)
1,Bacon,50,200 (400/100g),9 (17/100g),1 (1/100g),18 (35/100g)
2,Strawberries,100,32 (32/100g),1 (1/100g),8 (8/100g),0 (0/100g)
3,Blueberries,50,29 (57/100g),1 (1/100g),7 (14/100g),0 (0/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,357.81,404.00,12.91%
1,Protein,20.00,23.00,15.00%
2,Carb,20.96,18.00,14.12%
3,Fat,21.69,28.00,29.09%


In [127]:
predicted_data_2 = get_predicted_data("images/2.png")
formatted_predicted_data_2 = format_predicted_data(predicted_data_2, "images/2.png")
display(formatted_predicted_data_2["items"])
display(formatted_predicted_data_2["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,Roast Chicken Breast,300,570 (190/100g),90 (30/100g),0 (0/100g),24 (8/100g)
1,Lettuce,100,15 (15/100g),1 (1/100g),3 (3/100g),0 (0/100g)
2,Tomato,100,18 (18/100g),1 (1/100g),4 (4/100g),0 (0/100g)
3,Carrots,150,62 (41/100g),2 (1/100g),15 (10/100g),0 (0/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,655.99,665.00,1.37%
1,Protein,93.24,94.00,0.82%
2,Carb,23.34,22.00,5.74%
3,Fat,20.22,24.00,18.69%


In [10]:
percent_error_2 = get_percent_error("images/2.png")
percent_error_2

{'calories': {'actual': 655.99, 'predicted': 608, 'percent_error': 7.32},
 'protein': {'actual': 93.24, 'predicted': 63, 'percent_error': 32.43},
 'carb': {'actual': 23.34, 'predicted': 32, 'percent_error': 37.1},
 'fat': {'actual': 20.22, 'predicted': 33, 'percent_error': 63.2}}

In [128]:
predicted_data_3 = get_predicted_data("images/3.png")
formatted_predicted_data_3 = format_predicted_data(predicted_data_3, "images/3.png")
display(formatted_predicted_data_3["items"])
display(formatted_predicted_data_3["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,Watermelon,200,60 (30/100g),2 (1/100g),16 (8/100g),0 (0/100g)
1,Roasted Sweet Potato,100,90 (90/100g),2 (2/100g),20 (20/100g),0 (0/100g)
2,Blackberries,50,22 (43/100g),1 (1/100g),5 (10/100g),1 (1/100g)
3,Raspberries,50,26 (52/100g),1 (1/100g),6 (12/100g),1 (1/100g)
4,Pineapple,100,50 (50/100g),1 (1/100g),13 (13/100g),0 (0/100g)
5,Cottage Cheese (Low-fat),150,120 (80/100g),18 (12/100g),5 (3/100g),3 (2/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,351.89,368.00,4.58%
1,Protein,12.10,25.00,106.61%
2,Carb,57.50,65.00,13.04%
3,Fat,9.77,5.00,48.82%


In [67]:
predicted_data_3 = get_predicted_data("images/3.png")
predicted_data_3

{'items': [{'item': 'Watermelon',
   'estimated_weight_g': 200,
   'per_100g': {'calories': 30, 'protein': 1, 'carb': 8, 'fat': 0},
   'total': {'calories': 60, 'protein': 2, 'carb': 16, 'fat': 0}},
  {'item': 'Roasted sweet potato',
   'estimated_weight_g': 100,
   'per_100g': {'calories': 90, 'protein': 2, 'carb': 20, 'fat': 0},
   'total': {'calories': 90, 'protein': 2, 'carb': 20, 'fat': 0}},
  {'item': 'Pineapple',
   'estimated_weight_g': 100,
   'per_100g': {'calories': 50, 'protein': 0, 'carb': 13, 'fat': 0},
   'total': {'calories': 50, 'protein': 0, 'carb': 13, 'fat': 0}},
  {'item': 'Raspberries',
   'estimated_weight_g': 30,
   'per_100g': {'calories': 52, 'protein': 1, 'carb': 12, 'fat': 1},
   'total': {'calories': 16, 'protein': 0, 'carb': 4, 'fat': 0}},
  {'item': 'Blackberries',
   'estimated_weight_g': 30,
   'per_100g': {'calories': 43, 'protein': 1, 'carb': 10, 'fat': 1},
   'total': {'calories': 13, 'protein': 0, 'carb': 3, 'fat': 0}},
  {'item': 'Cottage cheese (l

In [129]:
predicted_data_4 = get_predicted_data("images/4.png")
formatted_predicted_data_4 = format_predicted_data(predicted_data_4, "images/4.png")
display(formatted_predicted_data_4["items"])
display(formatted_predicted_data_4["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,"Chicken Breast, Cooked, Roasted, or Baked (Skin Not Eaten)",150,248 (165/100g),47 (31/100g),0 (0/100g),6 (4/100g)
1,"Lettuce, Green Leaf, Raw",100,15 (15/100g),1 (1/100g),3 (3/100g),0 (0/100g)
2,"Onions, Red, Raw",50,20 (40/100g),1 (1/100g),5 (9/100g),0 (0/100g)
3,"Avocado, Raw, California",50,80 (160/100g),1 (2/100g),5 (9/100g),8 (15/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,184.99,363.00,96.23%
1,Protein,24.54,50.00,103.75%
2,Carb,8.90,13.00,46.07%
3,Fat,6.14,14.00,128.01%


In [66]:
predicted_data_4 = get_predicted_data("images/4.png")
predicted_data_4

{'items': [{'item': 'Chicken breast, cooked, roasted, or baked',
   'estimated_weight_g': 100,
   'per_100g': {'calories': 165, 'protein': 31, 'carb': 0, 'fat': 4},
   'total': {'calories': 165, 'protein': 31, 'carb': 0, 'fat': 4}},
  {'item': 'Lettuce, red leaf',
   'estimated_weight_g': 50,
   'per_100g': {'calories': 15, 'protein': 1, 'carb': 3, 'fat': 0},
   'total': {'calories': 8, 'protein': 1, 'carb': 2, 'fat': 0}},
  {'item': 'Lettuce, green leaf',
   'estimated_weight_g': 50,
   'per_100g': {'calories': 15, 'protein': 1, 'carb': 3, 'fat': 0},
   'total': {'calories': 8, 'protein': 1, 'carb': 2, 'fat': 0}},
  {'item': 'Onions, red, raw',
   'estimated_weight_g': 30,
   'per_100g': {'calories': 40, 'protein': 1, 'carb': 9, 'fat': 0},
   'total': {'calories': 12, 'protein': 0, 'carb': 3, 'fat': 0}},
  {'item': 'Avocado, raw, california',
   'estimated_weight_g': 50,
   'per_100g': {'calories': 160, 'protein': 2, 'carb': 9, 'fat': 15},
   'total': {'calories': 80, 'protein': 1, 'c

In [131]:
predicted_data_5 = get_predicted_data("images/5.png")
formatted_predicted_data_5 = format_predicted_data(predicted_data_5, "images/5.png")
display(formatted_predicted_data_5["items"])
display(formatted_predicted_data_5["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,Cantaloupe,120,41 (34/100g),1 (1/100g),10 (8/100g),0 (0/100g)
1,Bacon,30,120 (400/100g),5 (17/100g),0 (1/100g),11 (37/100g)
2,"Potatoes, roasted",80,74 (93/100g),2 (2/100g),17 (21/100g),0 (0/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,139.56,235.00,68.39%
1,Protein,5.40,8.00,48.15%
2,Carb,14.66,27.00,84.17%
3,Fat,6.64,11.00,65.66%


In [14]:
percent_error_5 = get_percent_error("images/5.png")
percent_error_5

{'calories': {'actual': 139.56, 'predicted': 300, 'percent_error': 114.96},
 'protein': {'actual': 5.4, 'predicted': 8, 'percent_error': 48.15},
 'carb': {'actual': 14.66, 'predicted': 36, 'percent_error': 145.57},
 'fat': {'actual': 6.64, 'predicted': 14, 'percent_error': 110.84}}

In [132]:
predicted_data_6 = get_predicted_data("images/6.png")
formatted_predicted_data_6 = format_predicted_data(predicted_data_6, "images/6.png")
display(formatted_predicted_data_6["items"])
display(formatted_predicted_data_6["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,Cooked Penne Pasta,100,131 (131/100g),5 (5/100g),27 (27/100g),1 (1/100g)
1,Cooked Beef Cubes,100,250 (250/100g),26 (26/100g),0 (0/100g),17 (17/100g)
2,Cooked Spinach,50,12 (23/100g),2 (3/100g),2 (4/100g),0 (0/100g)
3,Olive Oil (Assumed for Cooking),10,88 (884/100g),0 (0/100g),0 (0/100g),10 (100/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,950.40,481.00,49.39%
1,Protein,74.77,33.00,55.86%
2,Carb,54.13,29.00,46.43%
3,Fat,45.30,28.00,38.19%


In [15]:
percent_error_6 = get_percent_error("images/6.png")
percent_error_6

{'calories': {'actual': 950.4, 'predicted': 640, 'percent_error': 32.66},
 'protein': {'actual': 74.77, 'predicted': 35, 'percent_error': 53.19},
 'carb': {'actual': 54.13, 'predicted': 90, 'percent_error': 66.27},
 'fat': {'actual': 45.3, 'predicted': 20, 'percent_error': 55.85}}

In [134]:
predicted_data_7 = get_predicted_data("images/7.png")
formatted_predicted_data_7 = format_predicted_data(predicted_data_7, "images/7.png")
display(formatted_predicted_data_7["items"])
display(formatted_predicted_data_7["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,Scrambled eggs with cheese,70,140 (200/100g),12 (17/100g),1 (2/100g),11 (15/100g)
1,Fried potatoes (hash browns),100,150 (150/100g),2 (2/100g),20 (20/100g),8 (8/100g)
2,Cooked bacon,20,80 (400/100g),5 (27/100g),0 (1/100g),7 (35/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,284.69,370.00,29.97%
1,Protein,13.84,19.00,37.28%
2,Carb,14.42,21.00,45.63%
3,Fat,18.80,26.00,38.30%


In [16]:
percent_error_7 = get_percent_error("images/7.png")
percent_error_7

{'calories': {'actual': 284.69, 'predicted': 415, 'percent_error': 45.77},
 'protein': {'actual': 13.84, 'predicted': 20, 'percent_error': 44.51},
 'carb': {'actual': 14.42, 'predicted': 30, 'percent_error': 108.04},
 'fat': {'actual': 18.8, 'predicted': 25, 'percent_error': 32.98}}

In [137]:
predicted_data_8 = get_predicted_data("images/8.png")
formatted_predicted_data_8 = format_predicted_data(predicted_data_8, "images/8.png")
display(formatted_predicted_data_8["items"])
display(formatted_predicted_data_8["summary"])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [17]:
percent_error_8 = get_percent_error("images/8.png")
percent_error_8


{'calories': {'actual': 557.73, 'predicted': 495, 'percent_error': 11.25},
 'protein': {'actual': 25.77, 'predicted': 18, 'percent_error': 30.15},
 'carb': {'actual': 23.81, 'predicted': 35, 'percent_error': 47.0},
 'fat': {'actual': 42.54, 'predicted': 34, 'percent_error': 20.08}}

In [138]:
predicted_data_9 = get_predicted_data("images/9.png")
formatted_predicted_data_9 = format_predicted_data(predicted_data_9, "images/9.png")
display(formatted_predicted_data_9["items"])
display(formatted_predicted_data_9["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,Roasted Brussels Sprouts,150,57 (38/100g),4 (3/100g),12 (8/100g),0 (0/100g)
1,Mixed Green Salad,150,30 (20/100g),1 (1/100g),6 (4/100g),0 (0/100g)
2,Brown Rice,150,166 (111/100g),7 (5/100g),34 (23/100g),1 (1/100g)
3,Roasted Pork,150,319 (213/100g),30 (20/100g),0 (0/100g),21 (14/100g)
4,Cauliflower Rice,50,12 (25/100g),1 (2/100g),2 (5/100g),0 (0/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,583.38,584.00,0.11%
1,Protein,42.29,43.00,1.68%
2,Carb,52.42,54.00,3.01%
3,Fat,23.33,22.00,5.70%


In [18]:
percent_error_9 = get_percent_error("images/9.png")
percent_error_9

{'calories': {'actual': 583.38, 'predicted': 730, 'percent_error': 25.13},
 'protein': {'actual': 42.29, 'predicted': 50, 'percent_error': 18.23},
 'carb': {'actual': 52.42, 'predicted': 90, 'percent_error': 71.69},
 'fat': {'actual': 23.33, 'predicted': 35, 'percent_error': 50.02}}

In [139]:
predicted_data_10 = get_predicted_data("images/10.png")
formatted_predicted_data_10 = format_predicted_data(predicted_data_10, "images/10.png")
display(formatted_predicted_data_10["items"])
display(formatted_predicted_data_10["summary"])

,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,Broccoli,100,34 (34/100g),3 (3/100g),7 (7/100g),0 (0/100g)
1,"Sweet Potato, cooked, roasted, without skin, no salt",150,135 (90/100g),3 (2/100g),30 (20/100g),0 (0/100g)
2,"Blackberries, raw",50,22 (43/100g),1 (1/100g),5 (10/100g),1 (1/100g)
3,"Raspberries, raw",50,26 (52/100g),1 (1/100g),6 (12/100g),1 (1/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,163.86,217.00,32.43%
1,Protein,2.51,8.00,218.73%
2,Carb,28.49,48.00,68.48%
3,Fat,5.30,2.00,62.26%


In [19]:
percent_error_10 = get_percent_error("images/10.png")
percent_error_10

{'calories': {'actual': 163.86, 'predicted': 347, 'percent_error': 111.77},
 'protein': {'actual': 2.51, 'predicted': 11, 'percent_error': 338.25},
 'carb': {'actual': 28.49, 'predicted': 61, 'percent_error': 114.11},
 'fat': {'actual': 5.3, 'predicted': 8, 'percent_error': 50.94}}

In [148]:
predicted_data_11 = get_predicted_data("images/11.png")
formatted_predicted_data_11 = format_predicted_data(predicted_data_11, "images/11.png")
display(formatted_predicted_data_11["items"])
display(formatted_predicted_data_11["summary"])

I0000 00:00:1722555582.509106 6997231 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722555585.738187 6997231 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


,Item,Estimated Weight (g),Calories,Protein (g),Carbs (g),Fat (g)
0,Beef jerky,60,165 (275/100g),20 (33/100g),6 (10/100g),8 (13/100g)
1,Spinach,50,12 (23/100g),2 (3/100g),2 (4/100g),0 (0/100g)
2,Carrots,20,8 (41/100g),0 (1/100g),2 (10/100g),0 (0/100g)


,Nutrient,Actual,Predicted,Percent Error
0,Calories,428.67,185.00,56.84%
1,Protein,24.47,22.00,10.09%
2,Carb,4.90,10.00,104.08%
3,Fat,35.14,8.00,77.23%


In [23]:
percent_error_11 = get_percent_error("images/11.png")
percent_error_11

{'calories': {'actual': 428.67, 'predicted': 408, 'percent_error': 4.82},
 'protein': {'actual': 24.47, 'predicted': 36, 'percent_error': 47.12},
 'carb': {'actual': 4.9, 'predicted': 21, 'percent_error': 328.57},
 'fat': {'actual': 35.14, 'predicted': 21, 'percent_error': 40.24}}

In [157]:
predicted_data_12 = get_predicted_data("images/12.png")
formatted_predicted_data_12 = format_predicted_data(predicted_data_12, "images/12.png")
display(formatted_predicted_data_12["items"])
display(formatted_predicted_data_12["summary"])

I0000 00:00:1722557278.009596 6997231 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722557281.002690 6997231 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

In [24]:
percent_error_12 = get_percent_error("images/12.png")
percent_error_12

{'calories': {'actual': 506.36, 'predicted': 436, 'percent_error': 13.9},
 'protein': {'actual': 77.39, 'predicted': 39, 'percent_error': 49.61},
 'carb': {'actual': 16.41, 'predicted': 27, 'percent_error': 64.53},
 'fat': {'actual': 13.48, 'predicted': 24, 'percent_error': 78.04}}